# <span style="color:darkred"> QTM 151 Final Project </span>

<font size = "5">

Names: 

Section Number: 

Research question: "How does qualifying rank affect final results different across different tracks?"




# <span style="color:darkred"> I. Merging Data </span>

<font size = "5">
Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<font size = "5">
Import necessary tables

In [2]:
qualifying_raw  = pd.read_csv("raw_data/qualifying.csv")
results_raw     = pd.read_csv("raw_data/results.csv")
races_raw       = pd.read_csv("raw_data/races.csv")
circuits_raw    = pd.read_csv("raw_data/circuits.csv")

<font size = "5">
View relevant columns of each table and rename if necessary



<font size = "3">

Needed to rename position in qualifying and results table


In [3]:
races_raw[["raceId","circuitId"]]

,raceId,circuitId
0,1,1
1,2,2
2,3,17
3,4,3
4,5,4
...,...,...
1097,1116,69
1098,1117,32
1099,1118,18
1100,1119,80


In [5]:
circuits_raw[["circuitId","name"]]
#renaming name to circuit_name for clarity
circuits = circuits_raw.rename(columns = {'name':'circuit_name'})
circuits[["circuitId","circuit_name"]]

,circuitId,circuit_name
0,1,Albert Park Grand Prix Circuit
1,2,Sepang International Circuit
2,3,Bahrain International Circuit
3,4,Circuit de Barcelona-Catalunya
4,5,Istanbul Park
...,...,...
72,75,Autódromo Internacional do Algarve
73,76,Autodromo Internazionale del Mugello
74,77,Jeddah Corniche Circuit
75,78,Losail International Circuit


In [6]:
results_raw
#renaming position to final position (bc there is also a position column in qualifying table, but these are different positions)
results = results_raw.rename(columns = {'position':'final_position'})
results[["raceId","driverId", "final_position"]]

,raceId,driverId,final_position
0,18,1,1
1,18,2,2
2,18,3,3
3,18,4,4
4,18,5,5
...,...,...,...
25835,1096,854,16
25836,1096,825,17
25837,1096,1,18
25838,1096,849,19


In [7]:
qualifying_raw[["driverId","position"]]
qualifying = qualifying_raw.rename(columns = {'position':'qualifying_position'})
qualifying[["driverId", "qualifying_position"]]

,driverId,qualifying_position
0,1,1
1,9,2
2,5,3
3,13,4
4,2,5
...,...,...
9570,825,16
9571,842,17
9572,822,18
9573,848,19


In [8]:
#merging so we have final position as well as qualifying position 
positions_merge = pd.merge(results[["raceId","driverId", "final_position"]],
                           qualifying[["driverId","qualifying_position"]],
                           on = "driverId",
                           how = "left")

positions_merge


,raceId,driverId,final_position,qualifying_position
0,18,1,1,1.0
1,18,1,1,4.0
2,18,1,1,3.0
3,18,1,1,5.0
4,18,1,1,3.0
...,...,...,...,...
1657144,1096,4,\N,7.0
1657145,1096,4,\N,9.0
1657146,1096,4,\N,9.0
1657147,1096,4,\N,7.0


In [9]:
#merging the circuit IDs to each race 
positionrace_merge = pd.merge(positions_merge,
                              races_raw[["raceId","circuitId","year"]],
                              on = "raceId",
                              how = "left")

positionrace_merge


,raceId,driverId,final_position,qualifying_position,circuitId,year
0,18,1,1,1.0,1,2008
1,18,1,1,4.0,1,2008
2,18,1,1,3.0,1,2008
3,18,1,1,5.0,1,2008
4,18,1,1,3.0,1,2008
...,...,...,...,...,...,...
1657144,1096,4,\N,7.0,24,2022
1657145,1096,4,\N,9.0,24,2022
1657146,1096,4,\N,9.0,24,2022
1657147,1096,4,\N,7.0,24,2022


In [10]:
positionscircuits_merge = pd.merge(positionrace_merge,
                                   circuits[["circuitId","circuit_name"]],
                                   on = "circuitId",
                                   how = "left")

positionscircuits_merge

,raceId,driverId,final_position,qualifying_position,circuitId,year,circuit_name
0,18,1,1,1.0,1,2008,Albert Park Grand Prix Circuit
1,18,1,1,4.0,1,2008,Albert Park Grand Prix Circuit
2,18,1,1,3.0,1,2008,Albert Park Grand Prix Circuit
3,18,1,1,5.0,1,2008,Albert Park Grand Prix Circuit
4,18,1,1,3.0,1,2008,Albert Park Grand Prix Circuit
...,...,...,...,...,...,...,...
1657144,1096,4,\N,7.0,24,2022,Yas Marina Circuit
1657145,1096,4,\N,9.0,24,2022,Yas Marina Circuit
1657146,1096,4,\N,9.0,24,2022,Yas Marina Circuit
1657147,1096,4,\N,7.0,24,2022,Yas Marina Circuit


# <span style="color:darkred"> II. Data Manipulation </span>


## Data Cleaning: Drop non-numeric and Nan values 

### Looking at the codebook, the columns in our tables (columns that correspond to the variables we are using in our final merged table) that have null values are:

"position" in results table (which we renamed to final_position in our merged table)

"position" in qualifying table (which we renamed to qualifying_position in our merged table)

So, we have to drop na values for "final_position" and "qualifying_position"



#### First, we should extract a list of non-numeric variables in final_position, and change these values into Nan values so that we can drop all non-numeric and nan values in one step later

In [11]:
positionscircuits_merge

#first, extract list of non-numeric variables in final_position
subset = positionscircuits_merge.query("final_position.str.isnumeric() == False")
list_unique = pd.unique(subset["final_position"])
print(list_unique) # prints unique non-numeric values, which is only "\\N"

# replace the non-numeric values, in this case, "\\N' values with Nan
positionscircuits_merge["final_position"] = positionscircuits_merge["final_position"].replace(list_unique, np.nan)

['\\N']


#### Then, we should do the same for qualifying_position

In [12]:
subset1 = positionscircuits_merge.query("qualifying_position.str.isnumeric() == False")
list_unique1 = pd.unique(subset1["qualifying_position"])

AttributeError: Can only use .str accessor with string values!

#### This gives us the error: "Can only use .str accessor with string values!", which means that all of the non-numeric values are already Nan values, so we don't have to do any replacing

### Now, we make a final table that only has the .notna() values for final and qualifying positions 


In [13]:
finaltable = positionscircuits_merge.query('final_position.notna()').query('qualifying_position.notna()')
finaltable

,raceId,driverId,final_position,qualifying_position,circuitId,year,circuit_name
0,18,1,1,1.0,1,2008,Albert Park Grand Prix Circuit
1,18,1,1,4.0,1,2008,Albert Park Grand Prix Circuit
2,18,1,1,3.0,1,2008,Albert Park Grand Prix Circuit
3,18,1,1,5.0,1,2008,Albert Park Grand Prix Circuit
4,18,1,1,3.0,1,2008,Albert Park Grand Prix Circuit
...,...,...,...,...,...,...,...
1656803,1096,849,19,20.0,24,2022,Yas Marina Circuit
1656804,1096,849,19,20.0,24,2022,Yas Marina Circuit
1656805,1096,849,19,20.0,24,2022,Yas Marina Circuit
1656806,1096,849,19,16.0,24,2022,Yas Marina Circuit


## Data Cleaning: Recode 
### qualifying_position from float to int
### final_position from object to int
### circuitId from object to string

In [14]:
finaltable["qualifying_position"] = finaltable["qualifying_position"].astype(int)
finaltable["final_position"] = finaltable["final_position"].astype(int)
finaltable["circuitId"] = finaltable["circuitId"].astype(str)
finaltable

,raceId,driverId,final_position,qualifying_position,circuitId,year,circuit_name
0,18,1,1,1,1,2008,Albert Park Grand Prix Circuit
1,18,1,1,4,1,2008,Albert Park Grand Prix Circuit
2,18,1,1,3,1,2008,Albert Park Grand Prix Circuit
3,18,1,1,5,1,2008,Albert Park Grand Prix Circuit
4,18,1,1,3,1,2008,Albert Park Grand Prix Circuit
...,...,...,...,...,...,...,...
1656803,1096,849,19,20,24,2022,Yas Marina Circuit
1656804,1096,849,19,20,24,2022,Yas Marina Circuit
1656805,1096,849,19,20,24,2022,Yas Marina Circuit
1656806,1096,849,19,16,24,2022,Yas Marina Circuit


### Add a variable representing the difference in a driver's qualifying position and final race position
#### We can call this position_diff
abs(qualifying_position - final_position)


[13 11 14  6  7  3 16 17 15 10 12  9 18  5  4  8  2  1 19]
[28 25 26 27]


In [15]:
fn_diff = lambda x,y: x - y
finaltable["position_diff"] = list(map(fn_diff, finaltable["final_position"], finaltable["qualifying_position"]))
finaltable

,raceId,driverId,final_position,qualifying_position,circuitId,year,circuit_name,position_diff
0,18,1,1,1,1,2008,Albert Park Grand Prix Circuit,0
1,18,1,1,4,1,2008,Albert Park Grand Prix Circuit,-3
2,18,1,1,3,1,2008,Albert Park Grand Prix Circuit,-2
3,18,1,1,5,1,2008,Albert Park Grand Prix Circuit,-4
4,18,1,1,3,1,2008,Albert Park Grand Prix Circuit,-2
...,...,...,...,...,...,...,...,...
1656803,1096,849,19,20,24,2022,Yas Marina Circuit,-1
1656804,1096,849,19,20,24,2022,Yas Marina Circuit,-1
1656805,1096,849,19,20,24,2022,Yas Marina Circuit,-1
1656806,1096,849,19,16,24,2022,Yas Marina Circuit,3


In [19]:
finaltable_24 = finaltable.query("qualifying_position <= 24")
finaltable_24

,raceId,driverId,final_position,qualifying_position,circuitId,year,circuit_name,position_diff
0,18,1,1,1,1,2008,Albert Park Grand Prix Circuit,0
1,18,1,1,4,1,2008,Albert Park Grand Prix Circuit,-3
2,18,1,1,3,1,2008,Albert Park Grand Prix Circuit,-2
3,18,1,1,5,1,2008,Albert Park Grand Prix Circuit,-4
4,18,1,1,3,1,2008,Albert Park Grand Prix Circuit,-2
...,...,...,...,...,...,...,...,...
1656803,1096,849,19,20,24,2022,Yas Marina Circuit,-1
1656804,1096,849,19,20,24,2022,Yas Marina Circuit,-1
1656805,1096,849,19,20,24,2022,Yas Marina Circuit,-1
1656806,1096,849,19,16,24,2022,Yas Marina Circuit,3


#### How many instances did position_diff = 0?

In [27]:
sameposition = len(finaltable_24.query("position_diff == 0"))
sameposition

101141

### What proportion of all observations is this? (aka what percent of final position = qualifying position)

In [28]:
print(sameposition/len(finaltable_24))

0.07929797601640198


### Is this statistically significant? Lets run a simulation to see if a proportion of 7.925% of all results having the same final and qualifying position is signficant 

#### First, run a model that we can get random data from


The range of final and qualifying positions are from 1 to 24

Lets generate a vector of possible qualifying and final positions 


In [148]:
list_finalpos = (finaltable['final_position'].unique())
list_finalpos

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [149]:
list_qualifyingpos = (finaltable['qualifying_position'].unique())
list_qualifyingpos

array([ 1,  4,  3,  5,  2, 15, 10, 13,  9, 14, 16, 19, 12, 18, 11, 20,  6,
        7,  8, 21, 22, 17, 23, 24, 28, 25, 26, 27])


### Aggregate


In [ ]:
finaltable_agg = (finaltable_24.groupby('circuit_name')
                               .agg(mean_position_diff = ("abs_posdiff", "mean"), 
                                    sd_position_diff = ("abs_posdiff", "std")))

finaltable_agg

In [84]:
finaltablecircuits = (finaltable.groupby(["circuitId","circuit_name"])
                                .agg(count_races = ('raceId',len))
                                .sort_values(by = "count_races", ascending = False)
                                .iloc[0:5,:])
display(finaltablecircuits) 

,,count_races
circuitId,circuit_name,
9,Silverstone Circuit,73957
4,Circuit de Barcelona-Catalunya,70331
11,Hungaroring,69662
14,Autodromo Nazionale di Monza,69627
18,Autódromo José Carlos Pace,65251
